# Instalación de librerías

In [1]:
!pip install PyPDF2
!pip install stanza
!pip install sentiment-analysis-spanish
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.0/30.0 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00


# Importación de librerías

In [162]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import pandas as pd
import PyPDF2
import re
from itertools import combinations
import gdown
from gensim.models import KeyedVectors,Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import stanza
stanza.download('es')
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from langchain.text_splitter import CharacterTextSplitter
import unicodedata
from bs4 import BeautifulSoup
import requests
from sentence_transformers import SentenceTransformer, util
from prettytable import PrettyTable
from sentiment_analysis_spanish import sentiment_analysis
from transformers import BertTokenizer, BertForSequenceClassification,pipeline,BertModel
from deep_translator import GoogleTranslator
import csv
import warnings
warnings.filterwarnings("ignore")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: es (Spanish) ...
INFO:stanza:File exists: /root/stanza_resources/es/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Funciones

In [96]:
def traducir_texto(texto:str,idioma_entrada:str,idioma_salida:str)->str:
  """
  idiomdas aceptados: inglés (en) y español (es). Se indica en paréntesis el
  código ISO de cada idioma aceptado
  """
  texto_traducido = GoogleTranslator(source=idioma_entrada, target=idioma_salida).translate(texto)
  return texto_traducido

In [166]:
stopwords = [
  'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un',
  'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le',
  'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin',
  'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo',
  'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso',
  'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo',
  'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes',
  'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo',
  'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras',
  'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya',
  'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra',
  'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'es'
]

def elimina_puntuancion(texto):
  texto = re.sub(r'[^\w\s]','',texto)
  return texto

def elimina_stopwords(texto):
  palabras = texto.split()
  palabras_filtradas = [palabra for palabra in palabras if palabra.lower() not in stopwords]
  texto = ' '.join(palabras_filtradas)
  return texto

def minusculas(texto):
  texto = texto.lower()
  return texto

def elimina_acentos(texto):
  texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
  return texto

''' def tokenizacion(texto):
  nlp = es_core_news_sm.load()
  doc = nlp(texto)
  tokens = [token.text for token in doc]
  return tokens '''

def procesar_texto(texto):
  texto_sin_puntuacion = elimina_puntuancion(texto)
  texto_sin_stopwords = elimina_stopwords(texto_sin_puntuacion)
  texto_minuscula = minusculas(texto_sin_stopwords)
  texto_sin_acentos = elimina_acentos(texto_minuscula)
  return texto_sin_acentos

In [99]:
def normalizar_textos(descripciones,tema_pref):
  textos_normalizados = []
  for desc in descripciones:
    desc_normalizada = procesar_texto(desc)
    textos_normalizados.append(desc_normalizada)
  tema_pref_normalizado =  procesar_texto(tema_pref)
  textos_normalizados.append(tema_pref_normalizado)
  return textos_normalizados

# Clasificación del estado de ánimo

In [3]:
descripciones = [
    'estoy feliz porque estoy de vaciones',
    'por un lado está bueno que llueva porque estaba cansado de ir a la playa, pero por otro lado preferiría disfrutar del aire libre',
    'estoy peleado con todos mis familiares',
    'estoy muy triste porque nadie me quiere'
]

In [143]:
# Traducción del prompt del español al inglés usando el traductor de Google
def consultar_estado_animo()->str:
  """
  El usuario describe su estado de ánimo. Se retorta esta descripción traducida
  al inglés.
  """
  desc_estado_animo = input('¿Podrías describir cómo se siente hoy?:\t')
  return desc_estado_animo
def recomendacion_tipo_entretenimiento_segun_estado_animo(desc_estado_animo:str)->str:
  # Cargamos el tokenizador y el modelo
  mapeo_estrellas_tipo_entretenimiento = {
      '1 star': 'libro',
      '2 stars':'libro',
      '3 stars':'juego',
      '4 stars':'pelicula',
      '5 stars':'pelicula',
  }
  model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
  tokenizer = BertTokenizer.from_pretrained(model_name)
  model = BertForSequenceClassification.from_pretrained(model_name)
  # Creamos un pipeline de clasificación
  nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
  # Determinamos la etiqueta correspondiente
  result = nlp(desc_estado_animo)
  estrellas = result[0]['label']
  tipo_entr_recomendado = mapeo_estrellas_tipo_entretenimiento[estrellas]
  return tipo_entr_recomendado

# Ingreso de preferencias

In [127]:
# Pregunto por la temática de interés
def conocer_tema_interes():
  desc_tema = input('¿Podrías describir sobre qué temática en particual le gustaría buscar el contenido:\t')
  desc_ingles_tema = traducir_texto(texto=desc_tema,idioma_entrada='es',idioma_salida='en')
  return desc_ingles_tema

# Carga de los dataset

In [6]:
# dataset películas
!gdown 1vPzytA-f2RATVlh_uGm-rtp0sBeLQtbM

Downloading...
From: https://drive.google.com/uc?id=1vPzytA-f2RATVlh_uGm-rtp0sBeLQtbM
To: /content/IMDB-Movie-Data.csv
100% 309k/309k [00:00<00:00, 86.1MB/s]


In [7]:
# dataset videojuegos
!gdown 1vKriQRnGhNr1QU6YRZyvJNaJ2IWMknCx

Downloading...
From: https://drive.google.com/uc?id=1vKriQRnGhNr1QU6YRZyvJNaJ2IWMknCx
To: /content/bgg_database.csv
100% 1.83M/1.83M [00:00<00:00, 121MB/s]


## Dataset películas

In [8]:
# Leo el datast de películas
data_peliculas = []
with open('IMDB-Movie-Data.csv','r') as file:
  lineas = csv.reader(file)
  for linea in lineas:
    data_peliculas.append(linea)
encabezado_data_peliculas = data_peliculas.pop(0)

In [9]:
# Creo el dataframe de películas
df_peliculas = pd.DataFrame(data=data_peliculas,columns=encabezado_data_peliculas)
print(df_peliculas.info())
df_peliculas.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Rank                1000 non-null   object
 1   Title               1000 non-null   object
 2   Genre               1000 non-null   object
 3   Description         1000 non-null   object
 4   Director            1000 non-null   object
 5   Actors              1000 non-null   object
 6   Year                1000 non-null   object
 7   Runtime (Minutes)   1000 non-null   object
 8   Rating              1000 non-null   object
 9   Votes               1000 non-null   object
 10  Revenue (Millions)  1000 non-null   object
 11  Metascore           1000 non-null   object
dtypes: object(12)
memory usage: 93.9+ KB
None


,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7,485820,126.46,65
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40


In [10]:
df_peliculas['Genre'].unique()

array(['Action,Adventure,Sci-Fi', 'Adventure,Mystery,Sci-Fi',
       'Horror,Thriller', 'Animation,Comedy,Family',
       'Action,Adventure,Fantasy', 'Comedy,Drama,Music', 'Comedy',
       'Action,Adventure,Biography', 'Adventure,Drama,Romance',
       'Adventure,Family,Fantasy', 'Biography,Drama,History',
       'Animation,Adventure,Comedy', 'Action,Comedy,Drama',
       'Action,Thriller', 'Biography,Drama', 'Drama,Mystery,Sci-Fi',
       'Adventure,Drama,Thriller', 'Drama', 'Crime,Drama,Horror',
       'Action,Adventure,Drama', 'Drama,Thriller',
       'Action,Adventure,Comedy', 'Action,Horror,Sci-Fi',
       'Adventure,Drama,Sci-Fi', 'Action,Adventure,Western',
       'Comedy,Drama', 'Horror', 'Adventure,Drama,Fantasy',
       'Action,Crime,Thriller', 'Action,Crime,Drama',
       'Adventure,Drama,History', 'Crime,Horror,Thriller',
       'Drama,Romance', 'Comedy,Drama,Romance', 'Horror,Mystery,Thriller',
       'Crime,Drama,Mystery', 'Drama,Romance,Thriller',
       'Drama,History,T

In [11]:
for i in range(50):
  print(df_peliculas['Description'].loc[i],'\n\n')

A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe. 


Following clues to the origin of mankind, a team finds a structure on a distant moon, but they soon realize they are not alone. 


Three girls are kidnapped by a man with a diagnosed 23 distinct personalities. They must try to escape before the apparent emergence of a frightful new 24th. 


In a city of humanoid animals, a hustling theater impresario's attempt to save his theater with a singing competition becomes grander than he anticipates even as its finalists' find that their lives will never be the same. 


A secret government agency recruits some of the most dangerous incarcerated super-villains to form a defensive task force. Their first mission: save the world from the apocalypse. 


European mercenaries searching for black powder become embroiled in the defense of the Great Wall of China against a horde of monstrous creatures. 


A jazz pianist fal

## Dataset juegos

In [12]:
# Leo el datast de juegos
data_juegos = []
with open('/content/bgg_database.csv','r') as file:
  lineas = csv.reader(file)
  for linea in lineas:
    data_juegos.append(linea)
encabezado_data_juegos = data_juegos.pop(0)

In [13]:
# Creo el dataframe de películas
df_juegos = pd.DataFrame(data=data_juegos,columns=encabezado_data_juegos)
print(df_juegos.info())
df_juegos.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rank              1000 non-null   object
 1   game_name         1000 non-null   object
 2   game_href         1000 non-null   object
 3   geek_rating       1000 non-null   object
 4   avg_rating        1000 non-null   object
 5   num_voters        1000 non-null   object
 6   description       1000 non-null   object
 7   yearpublished     1000 non-null   object
 8   minplayers        1000 non-null   object
 9   maxplayers        1000 non-null   object
 10  minplaytime       1000 non-null   object
 11  maxplaytime       1000 non-null   object
 12  minage            1000 non-null   object
 13  avgweight         1000 non-null   object
 14  best_num_players  1000 non-null   object
 15  designers         1000 non-null   object
 16  mechanics         1000 non-null   object
 17  categories     

,rank,game_name,game_href,geek_rating,avg_rating,num_voters,description,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight,best_num_players,designers,mechanics,categories
0,1,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,8.415,8.6,46836.0,Brass: Birmingham is an economic strategy game...,2018,2,4,60,120,14,3.8776,"[{'min': 3, 'max': 4}]","['Gavan Brown', 'Matt Tolman', 'Martin Wallace']","['Hand Management', 'Income', 'Loans', 'Market...","['Age of Reason', 'Economic', 'Industry / Manu..."
1,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,8.377,8.53,53807.0,Pandemic Legacy is a co-operative campaign gam...,2015,2,4,60,60,13,2.8308,"[{'min': 4, 'max': 4}]","['Rob Daviau', 'Matt Leacock']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Environmental', 'Medical']"
2,3,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,8.349,8.59,62592.0,Gloomhaven is a game of Euro-inspired tactica...,2017,1,4,60,120,14,3.9132,"[{'min': 3, 'max': 3}]",['Isaac Childres'],"['Action Queue', 'Action Retrieval', 'Campaign...","['Adventure', 'Exploration', 'Fantasy', 'Fight..."
3,4,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,8.335,8.54,44728.0,"In Ark Nova, you will plan and design a modern...",2021,1,4,90,150,14,3.7653,"[{'min': 2, 'max': 2}]",['Mathias Wigge'],"['Action Queue', 'End Game Bonuses', 'Grid Cov...","['Animals', 'Economic', 'Environmental']"
4,5,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,8.24,8.6,24148.0,Twilight Imperium (Fourth Edition) is a game o...,2017,3,6,240,480,14,4.3173,"[{'min': 6, 'max': 6}]","['Dane Beltrami', 'Corey Konieczka', 'Christia...","['Action Drafting', 'Area-Impulse', 'Dice Roll...","['Civilization', 'Economic', 'Exploration', 'N..."


In [14]:
df_juegos['categories']

,categories
0,"['Age of Reason', 'Economic', 'Industry / Manu..."
1,"['Environmental', 'Medical']"
2,"['Adventure', 'Exploration', 'Fantasy', 'Fight..."
3,"['Animals', 'Economic', 'Environmental']"
4,"['Civilization', 'Economic', 'Exploration', 'N..."
...,...
995,"['Animals', 'Fantasy', 'Farming', 'Science Fic..."
996,"['City Building', 'Dice', 'Fantasy', 'Medieval']"
997,"['Bluffing', 'Card Game', 'Medieval', 'Negotia..."
998,['Abstract Strategy']


In [15]:
for i in range(50):
  #for category in df_juegos['categories'].loc[i]:
  print(df_juegos['categories'].loc[i])
  print('\n\n')

['Age of Reason', 'Economic', 'Industry / Manufacturing', 'Post-Napoleonic', 'Trains', 'Transportation']



['Environmental', 'Medical']



['Adventure', 'Exploration', 'Fantasy', 'Fighting', 'Miniatures']



['Animals', 'Economic', 'Environmental']



['Civilization', 'Economic', 'Exploration', 'Negotiation', 'Political', 'Science Fiction']



['Movies / TV / Radio theme', 'Novel-based', 'Political', 'Science Fiction']



['Economic', 'Environmental', 'Industry / Manufacturing', 'Science Fiction', 'Space Exploration', 'Territory Building']



['Fantasy', 'Fighting', 'Miniatures', 'Novel-based', 'Territory Building', 'Wargame']



['Civil War', 'Miniatures', 'Movies / TV / Radio theme', 'Science Fiction', 'Space Exploration', 'Wargame']



['Adventure', 'Exploration', 'Fantasy', 'Fighting', 'Miniatures']



['Environmental', 'Fantasy', 'Fighting', 'Mythology', 'Territory Building']



['Civilization', 'Economic', 'Miniatures', 'Science Fiction', 'Space Exploration', 'Territory Building

In [16]:
for i in range(50):
  print(df_juegos['description'].loc[i],'\n\n')

Brass: Birmingham is an economic strategy game sequel to Martin Wallace' 2007 masterpiece, Brass. Brass: Birmingham tells the story of competing entrepreneurs in Birmingham during the industrial revolution, between the years of 1770-1870.

It offers a very different story arc and experience from its predecessor. As in its predecessor, you must develop, build, and establish your industries and network, in an effort to exploit low or high market demands. The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs. VPs are counted at the end of each half for the canals, rails and established (flipped) industry tiles.

Each round, players take turns according to the turn order track, receiving two actions to perform any of the following actions (found in the original game):

1) Build - Pay required resources and place an industry tile.
2) Network - Add a rail / canal link, expanding your network.
3) Develop - I

## Dataset libros

In [17]:
# Creo una colección con los títulos de los libros disponibles
titulos_libros = []
url = f"https://www.gutenberg.org/browse/scores/top1000.php#books-last1"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
e_item_lista = soup.findAll('li')

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gutenberg.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [18]:
enlaces_libros = []
for e_item in e_item_lista:
  e_link = e_item.find('a')
  link = e_link.get('href')
  es_link_libro = re.search(r'ebooks/\d+',link)
  if es_link_libro: enlaces_libros.append(link)
enlaces_libros = enlaces_libros[:1000]

In [19]:
len(enlaces_libros)

1000

In [20]:
data_libros = []
for enlace in enlaces_libros:
  libro = {
      'Author':'',
      'Title':'',
      'Summary':'',
      'Subject':[]
  }
  url = f"https://www.gutenberg.org{enlace}"
  response = requests.get(url, verify=False)
  soup = BeautifulSoup(response.text, 'html.parser')
  e_contenedor_tabla = soup.find('div',{'id':'bibrec'})
  e_tabla = e_contenedor_tabla.find('table')
  e_filas = e_tabla.find_all('tr')
  for e_fila in e_filas:
    e_campo = e_fila.find('th')

    if e_campo is None or e_campo.text not in libro: continue
    campo = e_campo.text

    if (campo=='Author'):
      libro['Author'] = e_fila.find('td').text
    elif (campo=='Title'):
      libro['Title'] = e_fila.find('td').text
    elif (campo=='Summary'):
      libro['Summary'] = e_fila.find('td').text
    else:
      libro['Subject'].append(e_fila.find('td').text )
  data_libros.append(libro)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gutenberg.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gutenberg.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gutenberg.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpoo

In [21]:
# Creo el dataframe de películas
df_libros = pd.DataFrame(data=data_libros)
print(df_libros.info())
df_libros.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Author   1000 non-null   object
 1   Title    1000 non-null   object
 2   Summary  1000 non-null   object
 3   Subject  1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB
None


,Author,Title,Summary,Subject
0,"\nShelley, Mary Wollstonecraft, 1797-1851","\nFrankenstein; Or, The Modern Prometheus\n","\n""Frankenstein; Or, The Modern Prometheus"" by...","[\n\nScience fiction\n\n, \n\nHorror tales\n\n..."
1,"\nLü, Kun, 1536-1618",\n呻吟語\n,"\n""呻吟語"" by Kun Lü is a philosophical treatise ...",[\n\nConduct of life\n\n]
2,"\nMelville, Herman, 1819-1891","\nMoby Dick; Or, The Whale\n","\n""Moby Dick; Or, The Whale"" by Herman Melvill...","[\n\nWhaling -- Fiction\n\n, \n\nSea stories\n..."
3,"\nShakespeare, William, 1564-1616",\nRomeo and Juliet\n,"\n""Romeo and Juliet"" by William Shakespeare is...","[\n\nVendetta -- Drama\n\n, \n\nYouth -- Drama..."
4,"\nAusten, Jane, 1775-1817",\nPride and Prejudice\n,"\n""Pride and Prejudice"" by Jane Austen is a cl...","[\n\nEngland -- Fiction\n\n, \n\nYoung women -..."


## Dataset entretenimientos

In [137]:
# Recorte de datasets

df_peliculas_recortado = df_peliculas[['Title','Description']]
df_peliculas_recortado['tipo'] = 'pelicula'
df_peliculas_recortado.columns = ['nombre','descripcion','tipo']

df_juegos_recortado = df_juegos[['game_name','description']]
df_juegos_recortado['tipo'] = 'juego'
df_juegos_recortado.columns = ['nombre','descripcion','tipo']

df_libros_recortado = df_libros[['Title', 'Summary']]
df_libros_recortado['tipo'] = 'libro'
df_libros_recortado.columns = ['nombre','descripcion','tipo']

<ipython-input-137-7a585b1b2d0c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_peliculas_recortado['tipo'] = 'pelicula'
<ipython-input-137-7a585b1b2d0c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_juegos_recortado['tipo'] = 'juego'
<ipython-input-137-7a585b1b2d0c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [138]:
df_entretenimientos = pd.concat(objs=[df_peliculas_recortado,df_juegos_recortado,df_libros_recortado],ignore_index=True)
df_entretenimientos['tipo'].unique()

array(['pelicula', 'juego', 'libro'], dtype=object)

# Recomendaciones

In [ ]:
# Cargamos el modelo preentrenado multilingüe
modelo = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [160]:
def recomendar_segun_tipo_entretenimiento_y_tema_interes(tipo_entretenimiento:str,tema_interes:str)->str:
  df = df_entretenimientos[df_entretenimientos['tipo']==tipo_entretenimiento].reset_index()
  # Definimos una lista de oraciones
  oraciones = normalizar_textos(df['descripcion'],tema_interes)
  oraciones = [oracion for oracion in oraciones if len(oracion)<4500]
  # Codificamos las oraciones
  embeddings = modelo.encode(oraciones, convert_to_tensor=True)

  embeddings_dataset = embeddings[0:-1]
  embedding_query = embeddings[-1]

  puntuaciones_coseno = util.cos_sim(embeddings_dataset, embedding_query)

  puntuaciones_coseno_lista = []
  for i,puntuacion in enumerate(puntuaciones_coseno):
    resultado = {
        'index' : i,
        'puntuacion' : puntuacion
    }
    puntuaciones_coseno_lista.append(resultado)

  puntuaciones_coseno_lista_ord = sorted(puntuaciones_coseno_lista,key=lambda x:x['puntuacion'],reverse=True)

  for item in puntuaciones_coseno_lista_ord[:5]:
    i = item['index']
    registro = df.loc[i]
    titulo = registro['nombre']
    descripcion = traducir_texto(registro['descripcion'],'en','es')
    print('\t','titulo:',titulo)
    print('\t','descripción:',descripcion,'\n\n')


# Principal

In [167]:
for _ in range(1):
  estado_animo = consultar_estado_animo()
  tipo_entretenimiento_recomendado = recomendacion_tipo_entretenimiento_segun_estado_animo(estado_animo)
  print('le aconsejamos el siguiente tipo de entretenimiento: ',tipo_entretenimiento_recomendado)
  tema_interes = conocer_tema_interes()
  print('algunas de las opciones sugeridas son:')
  recomendar_segun_tipo_entretenimiento_y_tema_interes(tipo_entretenimiento_recomendado,tema_interes)
  print('\n')


¿Podrías describir cómo se siente hoy?:	feliz


Device set to use cpu


le aconsejamos el siguiente tipo de entretenimiento:  pelicula
¿Podrías describir sobre qué temática en particual le gustaría buscar el contenido:	historia de amor
algunas de las opciones sugeridas son:
	 titulo: Endless Love
	 descripción: La historia de una chica privilegiada y un chico carismático cuyo deseo instantáneo desencadena una historia de amor que se vuelve más temeraria cuando sus padres intentan mantenerlos separados. 


	 titulo: (500) Days of Summer
	 descripción: Una comedia romántica poco convencional sobre una mujer que no cree que exista el amor verdadero y el joven que se enamora de ella. 


	 titulo: The Bad Batch
	 descripción: Una historia de amor distópica en un páramo de Texas y ambientada en una comunidad de caníbales. 


	 titulo: Rules Don't Apply
	 descripción: La poco convencional historia de amor entre una aspirante a actriz, su decidido chofer y su jefe, el excéntrico multimillonario Howard Hughes. 


	 titulo: 20th Century Women
	 descripción: La histo